In [15]:
import pandas as pd
import numpy as np
import pickle

In [16]:
with open('distance_df_unique.pickle', 'rb') as f:
    distance_df = pickle.load(f)
distance_df.head()

,station,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
1,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5200fea1ee3,Salad Place,40.739980,-73.986648,0.759627
11,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5204ceb1ee3,Sushi Restaurant,40.738116,-73.987964,0.868923
14,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a52083e31ee3,Sushi Restaurant,40.737531,-73.991127,0.895991
17,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5208ae71ee3,Cupcake Shop,40.757140,-73.993508,0.479209
19,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a520b7e51ee3,Clothing Store,40.739744,-73.994964,0.773138


In [17]:
with open('Benson_full_analysis.pickle_v2', 'rb') as f:
    station_entries_df = pickle.load(f)
station_entries_df.head()

,STATION,ENTRIES,EXITS,mean_percentage
0,1 AV,108048.545455,121148.909091,1.383897
1,103 ST,173966.090909,108841.727273,2.228176
2,103 ST-CORONA,107435.272727,65010.454545,1.376042
3,104 ST,21531.545455,8188.272727,0.275778
4,110 ST,58447.818182,44527.000000,0.748606


In [18]:
distance_by_station = distance_df.groupby(by='station', as_index=False).count()

In [19]:
distance_df.head()

,station,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
1,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5200fea1ee3,Salad Place,40.739980,-73.986648,0.759627
11,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5204ceb1ee3,Sushi Restaurant,40.738116,-73.987964,0.868923
14,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a52083e31ee3,Sushi Restaurant,40.737531,-73.991127,0.895991
17,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a5208ae71ee3,Cupcake Shop,40.757140,-73.993508,0.479209
19,34 ST-PENN STA,40.750497,-73.990877,3fd66200f964a520b7e51ee3,Clothing Store,40.739744,-73.994964,0.773138


In [20]:
station_priorities = station_entries_df[['STATION', 'ENTRIES']].rename(columns={'ENTRIES': 'avg_entries', 'STATION': 'station'})
station_priorities.head()

,station,avg_entries
0,1 AV,108048.545455
1,103 ST,173966.090909
2,103 ST-CORONA,107435.272727
3,104 ST,21531.545455
4,110 ST,58447.818182


In [21]:
station_priorities = station_priorities.merge(distance_by_station, on='station')

In [22]:
station_priorities.head()

,station,avg_entries,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
0,125 ST,414684.000000,24,24,24,24,24,24,24
1,14 ST-UNION SQ,521808.545455,674,674,674,674,674,674,674
2,23 ST,546372.636364,515,515,515,515,515,515,515
3,34 ST-HERALD SQ,594431.181818,506,506,506,506,506,506,506
4,34 ST-PENN STA,784022.727273,498,498,498,498,498,498,498


In [23]:
station_priorities['avg_entries'] = station_priorities['avg_entries'].astype(int)

In [24]:
station_priorities.drop(['station_lat', 'station_lon', 'venueId', 'venueCategory', 'poi_lat', 'poi_lon'], axis=1, inplace = True)

In [25]:
station_priorities.rename(columns={'distance':'nearby_poi_count'},inplace=True)

In [26]:
station_priorities.head()

,station,avg_entries,nearby_poi_count
0,125 ST,414684,24
1,14 ST-UNION SQ,521808,674
2,23 ST,546372,515
3,34 ST-HERALD SQ,594431,506
4,34 ST-PENN STA,784022,498


In [29]:
station_priorities['entry_weight'] = station_priorities['avg_entries']/station_priorities['avg_entries'].sum()

In [30]:
station_priorities['poi_weight'] = station_priorities['nearby_poi_count']/station_priorities['nearby_poi_count'].sum()

In [32]:
station_priorities['total_weight'] = station_priorities['entry_weight'] * station_priorities['poi_weight']

In [36]:
station_priorities.sort_values('total_weight', inplace=True, ascending=False)

In [37]:
station_priorities

,station,avg_entries,nearby_poi_count,poi_weight,entry_weight,total_weight
4,34 ST-PENN STA,784022,498,0.132906,0.143164,0.019027
1,14 ST-UNION SQ,521808,674,0.179877,0.095283,0.017139
8,GRD CNTRL-42 ST,720653,483,0.128903,0.131593,0.016963
3,34 ST-HERALD SQ,594431,506,0.135041,0.108544,0.014658
2,23 ST,546372,515,0.137443,0.099769,0.013713
9,TIMES SQ-42 ST,463714,468,0.124900,0.084675,0.010576
5,42 ST-PORT AUTH,512094,387,0.103283,0.093510,0.009658
7,FULTON ST,496950,128,0.034161,0.090744,0.003100
6,86 ST,421655,64,0.017080,0.076995,0.001315
0,125 ST,414684,24,0.006405,0.075722,0.000485


In [ ]:
#distance_df.groupby(by=['venueCategory','station'], as_index=False).count().sort_values(['venueCategory','poi_lat'], ascending=[True,False])

In [38]:
with open('final_station_priorities.pickle', 'wb') as f:
    pickle.dump(station_priorities, f)
    